In [1]:
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentencesDataset,
    losses,
    CrossEncoder,
)
from torch.utils.data import DataLoader
from sentence_transformers import InputExample

import utils

In [2]:
import transformers
transformers.__version__

'4.18.0'

In [3]:
import sentence_transformers
sentence_transformers.__version__

'2.2.0'

In [4]:
dataset_path = utils.get_global_vars()["dataset_path"]
train_df = pd.read_csv(f"{dataset_path}/balanced_unhashed_train_df.csv")
test_df = pd.read_csv(f"{dataset_path}/balanced_unhashed_test_df.csv")

In [5]:
train_df = train_df.dropna()

In [6]:
print(any((train_df['answer1_label'] != 0) & (train_df['answer1_label'] != 1)))
print(any((train_df['answer2_label'] != 0) & (train_df['answer2_label'] != 1)))

False
False


In [7]:
index, row = next(train_df.iterrows())
row

comment_text      I've been searching for the answer for this fo...
comment_author                                             hans1193
comment_score                                                   194
answer1_text                          Religion must have the answer
answer1_author                                              Reedzit
answer1_score                                                     3
answer2_text      It's obviously tracks from a giant water tract...
answer2_author                                              adaminc
answer2_score                                                     1
subreddit                                                   science
answer1_label                                                     1
answer2_label                                                     0
Name: 0, dtype: object

In [8]:
from tqdm.notebook import tqdm

model = CrossEncoder("microsoft/deberta-base", num_labels=2)

train_examples = []
for index, row in tqdm(list(train_df.iterrows())):
    x = InputExample(
        texts = [row["comment_text"], row["answer1_text"]],
        label = int(row["answer1_label"])
    )
    train_examples.append(x)
    x = InputExample(
        texts = [row["comment_text"], row["answer2_text"]],
        label = int(row["answer2_label"])
    )
    train_examples.append(x)

train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)


Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.weight', 'pooler

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
model.fit(
    train_dataloader=train_dataloader,
    # evaluator=evaluator,
    
    epochs=5,
    warmup_steps=100,
)


/mnt/e/Lucas/_Estudos IME-USP/7_Semestre/Trabalho_de_Conclusão_de_Curso_MAC0499/Project/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 4.61 GiB already allocated; 0 bytes free; 4.80 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF